## Parse PCAP files and find out which IP address corresponds to which MAC address

When multiple IP addresses are assigned to the same MAC, it will report an warning.

In [11]:
import dpkt
import time
import socket
import os
import pandas as pd
import numpy as np
import struct
from IPython.display import display
from multiprocessing import Pool

### constants
g_dataDir = '../../iot_data/'
g_dataExtension = '.pcap'
g_testfile = '../../iot_data/16-09-27.pcap'
g_lanIpStartsWith = '192.168.'
g_showProgressPacketInterval = 1000000
g_poolSize = 10
g_device_info = {
    'd0:52:a8:00:67:5e': ['Smart Things', 'Wired'],
    '44:65:0d:56:cc:d3': ['Amazon Echo', 'Wireless'],
    '70:ee:50:18:34:43': ['Netatmo Welcome', 'Wireless'],
    'f4:f2:6d:93:51:f1': ['TP-Link Day Night Cloud camera', 'Wireless'],
    '00:16:6c:ab:6b:88': ['Samsung SmartCam', 'Wireless'],
    '30:8c:fb:2f:e4:b2': ['Dropcam', 'Wireless'],
    '00:62:6e:51:27:2e': ['Insteon Camera', 'Wired '],
    'e8:ab:fa:19:de:4f': ['Unknown', 'Wireless'],
    '00:24:e4:11:18:a8': ['Withings Smart Baby Monitor', 'Wired'],
    'ec:1a:59:79:f4:89': ['Belkin Wemo switch', 'Wireless'],
    '50:c7:bf:00:56:39': ['TP-Link Smart plug', 'Wireless'],
    '74:c6:3b:29:d7:1d': ['iHome', 'Wireless'],
    'ec:1a:59:83:28:11': ['Belkin wemo motion sensor', 'Wireless'],
    '18:b4:30:25:be:e4': ['NEST Protect smoke alarm', 'Wireless'],
    '70:ee:50:03:b8:ac': ['Netatmo weather station', 'Wireless'],
    '00:24:e4:1b:6f:96': ['Withings Smart scale', 'Wireless'],
    '74:6a:89:00:2e:25': ['Blipcare Blood Pressure meter', 'Wireless'],
    '00:24:e4:20:28:c6': ['Withings Aura smart sleep sensor', 'Wireless'],
    'd0:73:d5:01:83:08': ['Light Bulbs LiFX Smart Bulb', 'Wireless'],
    '18:b7:9e:02:20:44': ['Triby Speaker', 'Wireless'],
    'e0:76:d0:33:bb:85': ['PIX-STAR Photo-frame', 'Wireless'],
    '70:5a:0f:e4:9b:c0': ['HP Printer', 'Wireless'],
    '08:21:ef:3b:fc:e3': ['Samsung Galaxy Tab', 'Wireless'],
    '30:8c:fb:b6:ea:45': ['Nest Dropcam', 'Wireless'],
    '40:f3:08:ff:1e:da': ['Android Phone', 'Wireless'],
    '74:2f:68:81:69:42': ['Laptop', 'Wireless'],
    'ac:bc:32:d4:6f:2f': ['MacBook', 'Wireless'],
    'b4:ce:f6:a7:a3:c2': ['Android Phone', 'Wireless'],
    'd0:a6:37:df:a1:e1': ['IPhone', 'Wireless'],
    'f4:5c:89:93:cc:85': ['MacBook/Iphone', 'Wireless'],
    '14:cc:20:51:33:ea': ['TPLink Router Bridge LAN (Gateway)', 'Wired']
}

### initialization
start = time.time()
dataFiles = []
for fileName in os.listdir(g_dataDir):
    if fileName.endswith(g_dataExtension):
        dataFiles.append(os.path.join(g_dataDir, fileName))
pool = Pool(g_poolSize)

### parse pcap files and return MAC/IP dict
def parsePcap(fileName):
    counter = 0
    devices = {}
    file = open(fileName,'rb')
    for ts, pkt in dpkt.pcapng.Reader(file):
        counter += 1
                    
        eth = dpkt.ethernet.Ethernet(pkt)
        if eth.type != dpkt.ethernet.ETH_TYPE_ARP:
            continue
            
        arp = eth.data
        ip = socket.inet_ntoa(arp.spa)
        if not ip.startswith(g_lanIpStartsWith):
            continue
            
        mac = "%02x:%02x:%02x:%02x:%02x:%02x" % struct.unpack("BBBBBB",arp.sha)
        if ip not in devices:
            devices[ip] = mac
        else:
            if devices[ip] != mac:
                print('Warning: ip=%s, mac1=%s, mac2=%s' % (ip, devices[ip], mac) )
    
    print('Finished parsing: ' + fileName + '\n\n')
    return devices

### send job to workers
devicesArray = pool.map(parsePcap, dataFiles)

### merge results
devices = {}
for d in devicesArray:
    for key, value in d.items():
        if key not in devices:
            devices[key] = value
        if devices[key] != value:
            print('Warning: ip=%s, mac1=%s, mac2=%s' % (key, devices[key], value))

### print result
print ("Time elapsed: " + str(time.time() - start))
for key, value in sorted(devices.items()):
    if value in g_device_info:
        print(key, value, g_device_info[value][0], g_device_info[value][1])
    else:
        print(key, value)
        

Finished parsing: ../../iot_data/16-09-25.pcap


Finished parsing: ../../iot_data/16-10-02.pcap


Finished parsing: ../../iot_data/16-10-08.pcap


Finished parsing: ../../iot_data/16-09-26.pcap


Finished parsing: ../../iot_data/16-09-29.pcap


Finished parsing: ../../iot_data/16-09-30.pcap


Finished parsing: ../../iot_data/16-09-24.pcap


1000000 packets processed
Finished parsing: ../../iot_data/16-10-11.pcap


1000000 packets processed
Finished parsing: ../../iot_data/16-10-07.pcap


1000000 packets processed
Finished parsing: ../../iot_data/16-09-28.pcap


Finished parsing: ../../iot_data/16-09-27.pcap


Finished parsing: ../../iot_data/16-10-03.pcap


Finished parsing: ../../iot_data/16-10-09.pcap


Finished parsing: ../../iot_data/16-10-06.pcap


Finished parsing: ../../iot_data/16-10-01.pcap


1000000 packets processed
Finished parsing: ../../iot_data/16-10-05.pcap


Finished parsing: ../../iot_data/16-10-10.pcap


1000000 packets processed
Finished parsing: ../../iot_data/16-1